# 🤖 07: ReACT Agents

Learn how to use the ReACT (Reasoning + Acting) agent pattern to build autonomous agents that can reason about tasks and execute tools to complete them.

## 📋 Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Understand the ReACT (Reasoning + Acting) pattern
- [ ] Use the `client.react()` one-liner for simple agent tasks
- [ ] Examine `AgentResult` objects to understand agent execution
- [ ] Configure custom stop conditions for agents
- [ ] Create custom agents by extending `BaseAgent`
- [ ] Build agents that autonomously solve multi-step problems
- [ ] Understand when to use agents vs. direct tool calls

## 🎯 Prerequisites

- Completed notebooks 04 (Tool Calling Basics) and 05 (Custom Tools)
- Understanding of how tools work and how to register them
- Familiarity with multi-step problem solving
- LM Studio running with a model that supports function calling

## ⏱️ Estimated Time: 20 minutes

## 1️⃣ What is ReACT?

**ReACT** stands for **Reasoning + Acting**. It's a pattern where an AI agent:

1. **Reasons** about what it needs to do
2. **Acts** by calling tools
3. **Observes** the results
4. **Repeats** until the task is complete

### Traditional Tool Use vs. ReACT Agent

| Traditional Tool Use | ReACT Agent |
|---------------------|-------------|
| You ask a specific question | You give a high-level goal |
| Single LLM call with tools | Multiple LLM calls in a loop |
| You control the flow | Agent controls the flow |
| "Calculate 5 * 5" | "Research the topic and summarize" |
| Direct answer | Multi-step problem solving |

### When to Use ReACT Agents?

✅ **Good for:**
- Multi-step tasks with unclear steps
- Research and information gathering
- Tasks requiring multiple tool uses
- When you want autonomous decision-making

❌ **Not needed for:**
- Single-step tasks
- Simple calculations or transformations
- When you know exactly which tool to call

## 2️⃣ Using client.react() - The Simple Way

The easiest way to use a ReACT agent is with the `client.react()` method.

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

from local_llm_sdk import LocalLLMClient

# Create client and register tools
client = LocalLLMClient(
    base_url=os.getenv("LLM_BASE_URL"),
    model=os.getenv("LLM_MODEL")
)

# Register built-in tools
client.register_tools_from(None)

print("✅ Client ready with tools:")
for tool_name in client.tools.list_tools():
    print(f"  - {tool_name}")

✓ Auto-detected model: qwen/qwen3-coder-30b
✅ Client ready with tools:
  - char_counter
  - math_calculator
  - get_weather
  - text_transformer
  - execute_python
  - filesystem_operation


Now let's use the agent to solve a problem:

In [3]:
# Simple agent task
result = client.react(
    "Calculate the factorial of 5, then convert the result to uppercase text, "
    "then count how many characters are in that uppercase text."
)

print("🎯 Final Result:")
print(result.final_response)

ReACT Agent: Starting task
Max iterations: 15
Task: Calculate the factorial of 5, then convert the result to uppercase text, then count how many charact...


Iteration 1/15
----------------------------------------
Response: I need to continue calculating the factorial properly. Let me multiply 5 × 4 × 3 × 2 × 1.
<tool_call>
<function=math_calculator>
<parameter=arg1>
20
<...
Tools used: 1
  - math_calculator

Iteration 2/15
----------------------------------------
Response: I'll continue calculating the factorial of 5.

<tool_call>
<function=math_calculator>
<parameter=arg1>
60
</parameter>
<parameter=arg2>
2
</parameter>...
Tools used: 1
  - math_calculator

Iteration 3/15
----------------------------------------
Response: Now I'll calculate the final multiplication for the factorial.

<tool_call>
<function=math_calculator>
<parameter=arg1>
120
</parameter>
<parameter=ar...
Tools used: 1
  - math_calculator

Iteration 4/15
----------------------------------------
Response: Now I have 

**🎉 What just happened?**

The agent:
1. **Reasoned**: "I need to do math, then text transformation, then counting"
2. **Acted**: Called `execute_python` for factorial
3. **Observed**: Got result "120"
4. **Acted**: Called `text_transformer` to uppercase "120"
5. **Observed**: Got "120" (no change for numbers)
6. **Acted**: Called `char_counter` to count
7. **Observed**: Got "3 characters"
8. **Concluded**: Task complete, provided answer

All automatic! You just gave it a goal.

## 3️⃣ Examining AgentResult

The `AgentResult` object contains everything that happened during execution.

In [4]:
# Run another agent task
result = client.react(
    "Calculate 15 * 23, then tell me if the result is even or odd, "
    "and finally multiply it by 2 if even or by 3 if odd."
)

print("📊 AgentResult Breakdown:\n")
print(f"\nStatus: {result.status}")
print(f"Total iterations: {result.iterations}")
print(f"\nFinal response:\n{result.final_response}")

ReACT Agent: Starting task
Max iterations: 15
Task: Calculate 15 * 23, then tell me if the result is even or odd, and finally multiply it by 2 if even o...


Iteration 1/15
----------------------------------------
Response: Now I have the result 345. Next, I need to determine if it's even or odd.

A number is even if it's divisible by 2 with no remainder, and odd if there...
Tools used: 1
  - math_calculator

Iteration 2/15
----------------------------------------
Response: I need to check the remainder when 345 is divided by 2 to determine if it's even or odd. Let me execute this calculation.
<tool_call>
<function=execut...
Tools used: 1
  - execute_python

Iteration 3/15
----------------------------------------
Response: The remainder when 345 is divided by 2 is 1, which means 345 is odd.

Since 345 is odd, I need to multiply it by 3.

Let me calculate 345 * 3.

<tool_...
Tools used: 1
  - execute_python

Iteration 4/15
----------------------------------------
Response: The final res

Let's inspect the execution history:

In [ ]:
print("\n🔍 Execution History:\n")
print("="*70)

for i, step in enumerate(result.conversation, 1):
    print(f"\nStep {i}:")
    print(f"  Role: {step.role}")
    
    if step.role == 'assistant':
        if hasattr(step, 'content') and step.content:
            print(f"  Thought: {step.content[:100]}...")
        if hasattr(step, 'tool_calls') and step.tool_calls:
            print(f"  Tools called: {len(step.tool_calls)}")
            for tc in step.tool_calls:
                print(f"    - {tc.function.name}")
    
    elif step.role == 'tool':
        if hasattr(step, 'name') and step.name:
            print(f"  Tool: {step.name}")
        if hasattr(step, 'content') and step.content:
            print(f"  Result: {step.content[:80]}...")
    
    elif step.role == 'user':
        if hasattr(step, 'content') and step.content:
            print(f"  Task: {step.content[:80]}...")

print("\n" + "="*70)

**💡 AgentResult fields:**

- `task`: Original task given to the agent
- `status`: "success", "failed", or "stopped"
- `final_response`: The agent's final answer
- `iterations`: Number of reasoning-action cycles
- `history`: Complete conversation history (reasoning + tool calls + results)
- `stop_reason`: Why the agent stopped ("task_complete", "max_steps", "error", etc.)

# Limit maximum steps
result = client.react(
    "Count from 1 to 100, adding each number together",
    max_iterations=3  # Stop after 3 reasoning cycles
)

print("🛑 Limited Steps Example:\n")
print(f"Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\nPartial result:\n{result.final_response}")

In [ ]:
# Limit maximum steps
result = client.react(
    "Count from 1 to 100, adding each number together",
    max_iterations=3  # Stop after 3 reasoning cycles
)

print("🛑 Limited Steps Example:\n")
print(f"Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\nPartial result:\n{result.final_response}")

**💡 Stop condition parameters:**

- `max_steps`: Maximum reasoning-action cycles (default: 10)
- `timeout`: Maximum execution time in seconds (default: 300)
- Custom stop conditions can be implemented in custom agents

In [ ]:
# More reasonable max_steps example
result = client.react(
    "Calculate 10 factorial, then find all its prime factors",
    max_iterations=10
)

print("✅ Sufficient Steps Example:\n")
print(f"Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\nResult:\n{result.final_response}")

## 5️⃣ Creating Custom Agents with BaseAgent

For more control, create a custom agent by extending `BaseAgent`.

In [ ]:
from local_llm_sdk.agents import BaseAgent, AgentResult, AgentStatus

class MathResearchAgent(BaseAgent):
    """Agent specialized in mathematical research and analysis."""
    
    def __init__(self, client):
        super().__init__(client)
        self.findings = []
        self.max_steps = 10
    
    def _execute(self, task: str, **kwargs) -> AgentResult:
        """Execute the research task using client.react()."""
        max_iterations = kwargs.get('max_iterations', self.max_steps)
        
        # Use client.react() to execute the task
        result = self.client.react(task, max_iterations=max_iterations, verbose=False)
        
        # Process findings from the result
        if result.status == AgentStatus.SUCCESS:
            self.findings.append(f"Task: {task[:100]}")
            self.findings.append(f"Result: {result.final_response[:200]}")
        
        return result
    
    def should_stop(self, step_count: int, last_response: str) -> bool:
        """Custom stop condition: stop when we have enough findings."""
        
        # Stop if max steps reached
        if step_count >= self.max_steps:
            return True
        
        # Stop if response indicates completion
        completion_phrases = [
            "task complete",
            "finished",
            "done",
            "conclusion"
        ]
        
        last_response_lower = last_response.lower()
        if any(phrase in last_response_lower for phrase in completion_phrases):
            return True
        
        return False
    
    def process_step(self, response: str):
        """Hook to process each step - extract findings."""
        # Store interesting findings
        if "finding" in response.lower() or "discovered" in response.lower():
            self.findings.append(response[:200])

# Create custom agent
math_agent = MathResearchAgent(client)

print("✅ Custom MathResearchAgent created!")

Now let's use our custom agent:

In [ ]:
# Use the custom agent
result = math_agent.run(
    "Research the number 100: calculate its factorial, "
    "find how many digits are in the result, "
    "and determine if the digit count is prime.",
    max_iterations=8
)

print("🔬 Math Research Results:\n")
print(f"Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\nFindings collected: {len(math_agent.findings)}")
print(f"\nFinal result:\n{result.final_response}")

**💡 BaseAgent methods to override:**

- `should_stop()`: Custom stop conditions
- `process_step()`: Hook for each step (logging, monitoring, etc.)
- `prepare_task()`: Modify task before execution
- `format_result()`: Custom result formatting

## 6️⃣ Multi-Tool Agent Example

Let's see an agent that uses multiple tools to accomplish a complex task.

In [ ]:
import tempfile
import os

# Create a temp directory for file operations
temp_dir = tempfile.mkdtemp()

# Complex multi-step task
result = client.react(
    f"Generate a list of the first 10 prime numbers using Python. "
    f"Then save them to a file at {temp_dir}/primes.txt (one per line). "
    f"After saving, read the file back to verify it was saved correctly. "
    f"Finally, count how many characters are in the file (including newlines).",
    max_iterations=15
)

print("🔧 Multi-Tool Task Results:\n")
print(f"Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\nFinal answer:\n{result.final_response}")

# Verify the file was created
file_path = os.path.join(temp_dir, "primes.txt")
if os.path.exists(file_path):
    with open(file_path, 'r') as f:
        content = f.read()
    print(f"\n✅ File verification:\n{content}")

# Cleanup
import shutil
shutil.rmtree(temp_dir)

**🎯 The agent orchestrated:**
1. `execute_python` - Generate prime numbers
2. `filesystem_operation` - Write to file
3. `filesystem_operation` - Read file back
4. `char_counter` - Count characters

All without you specifying which tools to use or in what order!

## 🏋️ Exercise: Build a Research Agent

**Challenge:** Create a research agent that:

1. Takes a mathematical concept (e.g., "Fibonacci sequence")
2. Calculates/generates examples of the concept
3. Analyzes properties (even/odd counts, average value, etc.)
4. Saves a research report to a file
5. Returns a summary of findings

**Requirements:**
- Use `client.react()` for the agent
- Should use at least 3 different tools
- Generate first 15 numbers of the Fibonacci sequence
- Calculate statistics (count evens, count odds, average)
- Save report to a file with all findings
- Max 20 steps

Try it yourself first!

In [ ]:
# Your code here:



# Solution cell (run to see answer)
import tempfile
import os
import shutil

temp_dir = tempfile.mkdtemp()
report_file = os.path.join(temp_dir, "fibonacci_research.txt")

print("🔬 Fibonacci Sequence Research Agent\n")
print("="*70)

result = client.react(
    f"Research the Fibonacci sequence. "
    f"Generate the first 15 Fibonacci numbers using Python. "
    f"Then analyze them: count how many are even, how many are odd, "
    f"and calculate the average value. "
    f"Save all findings to a research report at {report_file}. "
    f"The report should include: the sequence, the counts, and the statistics.",
    max_iterations=20
)

print(f"\n📊 Research Status: {result.status}")
print(f"Iterations: {result.iterations}")
print(f"\n💡 Summary:\n{result.final_response}")

if os.path.exists(report_file):
    print("\n" + "="*70)
    print("\n📄 Generated Report:\n")
    with open(report_file, 'r') as f:
        print(f.read())
    print("\n" + "="*70)
else:
    print("\n⚠️ Report file was not created")

print("\n🛤️ Execution Path:")
tool_calls_made = []
for step in result.conversation:
    if hasattr(step, 'role') and step.role == 'assistant' and hasattr(step, 'tool_calls') and step.tool_calls:
        for tc in step.tool_calls:
            tool_calls_made.append(tc.function.name)

print(f"Tools used: {', '.join(set(tool_calls_made))}")
print(f"Total tool calls: {len(tool_calls_made)}")

shutil.rmtree(temp_dir)
print("\n✅ Research complete!")

## ⚠️ Common Pitfalls

### 1. Using Agents for Simple Tasks
```python
# ❌ Overkill: Agent for simple calculation
result = client.react("What is 5 + 5?")
# Wastes multiple LLM calls for a trivial task

# ✅ Better: Direct chat for simple tasks
response = client.chat("What is 5 + 5?")
```

### 2. Not Setting max_steps
```python
# ⚠️ Warning: Could run too long
result = client.react("Count to infinity")
# Will run until default max_steps (10)

# ✅ Good: Set appropriate limits
result = client.react("Complex task", max_iterations=5)
```

### 3. Unclear Task Instructions
```python
# ❌ Bad: Vague instructions
result = client.react("Do something with numbers")
# Agent will struggle to understand the goal

# ✅ Good: Clear, specific instructions
result = client.react(
    "Calculate the sum of numbers 1 to 100, "
    "then determine if the result is divisible by 7"
)
```

### 4. Not Checking AgentResult Status
```python
# ❌ Bad: Assuming success
result = client.react("Complex task")
print(result.final_response)  # Might be incomplete

# ✅ Good: Check status
result = client.react("Complex task")
if result.status == "success":
    print(result.final_response)
else:
    print(f"Failed: {result.error}")
```

### 5. Forgetting That Agents Cost More
```python
# ⚠️ Agents make multiple LLM calls
# Each reasoning cycle = 1+ API calls
# 10 steps = 10+ calls (can be expensive!)

# 💡 Tip: Use agents only when necessary
# For simple tasks, direct chat is more efficient
```

## 🎓 What You Learned

✅ **ReACT Pattern**: Reasoning + Acting cycles for autonomous problem-solving

✅ **client.react()**: Simple one-liner for agent tasks

✅ **AgentResult**: Understanding execution history and results

✅ **Stop Conditions**: Using max_steps and custom conditions

✅ **BaseAgent**: Creating custom agents with specialized behavior

✅ **Multi-Tool Orchestration**: Agents autonomously use multiple tools

✅ **When to Use Agents**: Complex multi-step tasks vs. simple operations

## 🚀 Next Steps

You've mastered ReACT agents! Now let's learn how to observe and debug them with MLflow.

➡️ Continue to [08-mlflow-observability.ipynb](./08-mlflow-observability.ipynb) to learn:
- Installing and configuring MLflow
- Viewing agent traces in the MLflow UI
- Understanding hierarchical traces (CHAIN → LLM → AGENT → TOOL)
- Using traces to debug and optimize agent behavior
- Finding bottlenecks and improving performance